In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence

class SentimentAnalysis(nn.Module):
    def __init__(
            self,
            vocab_size: int,
            hidden_size: int,
            embedding_size: int,
        ):
        self._hidden_size = hidden_size
        self._embedding_size = embedding_size

        self.emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size)

        self.gru = nn.GRU(
            input_size=self._embedding_size,
            hidden_size=self._hidden_size,
        )

        inner_size = 100

        self.seq = nn.Sequential(
            nn.Linear(hidden_size, inner_size),
            nn.ReLU(),
            nn.Linear(inner_size, 2),
        )

    def forward(self, x: torch.Tensor, lengths: torch.Tensor):
        # x (n_batch, seq)

        # apply embeddings
        x = self.emb(x)
        # x (n_batch, seq, emb_dim)

        x.transpose(0, 1)
        # x (seq, n_batch, emb_dim)

        x = pack_padded_sequence(x, lengths)

        # take last hidden layer output
        _, h_n = self.gru(x)

        return self.seq(h_n[-1, :, :])